# WhatsAppPush Notification Notebook
Arunasish Ghosh

In [2]:
import requests, json, uuid, time, csv, io

BASE_URL = 'https://e0cfn0j28h.execute-api.us-east-1.amazonaws.com/dev'
CREATE_NOTIFICATION_URL = BASE_URL + '/notification'
LIST_NOTIFICATION_URL = BASE_URL + '/notification/{user_id}'
CREATE_TEMPLATE_URL = BASE_URL + '/templates'
GET_SIGNED_URL = BASE_URL + '/upload-url/{user_id}'

USERID = 'jupyter_' + str(uuid.uuid4())[:8]

print('user_id:', USERID)

user_id: jupyter_462d26fc


## ** Create Notification

### Invalid Request: missing message/message_template_id

In [ ]:
payload = dict(recipient = '8080808080', user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Invalid Request: missing recipient/recipient_list_file

In [ ]:
payload = dict(message = 'Best Wishes', user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Invalid Request: missing user_id

In [ ]:
payload = dict(message = 'Best Wishes', recipient = '8080808080')

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Invalid Request: incorrect message_template_id

In [ ]:
payload = dict(message_template_id = 'randomId', recipient = '8080808080', user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Invalid Request: incorrect recipient_list_file

In [ ]:
payload = dict(message = 'Best Wishes', recipient_list_file = 'randomFile', user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Valid Request: with message, single recipient

In [ ]:
payload = dict(message = 'Best Wishes', recipient = '8080808080', user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Create a template for later use

In [ ]:
payload = dict(templateName = 'Template 1',
               messageText = 'Best Wishes from serverless.',
               user_id = USERID,
               idempotent_key = USERID + '_UNIQ123')

response = requests.post(CREATE_TEMPLATE_URL, data = json.dumps(payload))
rspJson = response.json()
templateId = rspJson['template_id']

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))
print('templateId extracted:', templateId)

### Valid Request: with message_template_id, single recipient

In [ ]:
payload = dict(message_template_id = templateId, recipient = '8080808080', user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Get signed url to upload csv

In [ ]:
csvFileName = 'NewFile_' + str(int(time.time())) + '.csv'
payload = dict(file_name = csvFileName)

response = requests.post(GET_SIGNED_URL.format(user_id = USERID), data = json.dumps(payload))
rspJson = response.json()
uploadUrl = rspJson['url']

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))
print('Copied upload URL')

### Prepare and upload csv using pre signed url

In [ ]:
from requests import Request, Session
 
rows = [['Numbers'], ['9876543210'], ['8765432109'], ['7654321098'], ['6543210987']]

buffer = io.StringIO()
writer = csv.writer(buffer)

for row in rows:
    writer.writerow(row)

content = buffer.getvalue()
content.encode('utf-8')

session = Session()
request = Request('PUT', uploadUrl, data = format(content)).prepare()
response = session.send(request)

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', response.text)

### Valid Request: with message_template_id, bulk recipient list file

In [ ]:
payload = dict(message_template_id = templateId, recipient_list_file = csvFileName, user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

## ** List Notifications for user 

### Valid user id

In [4]:
response = requests.get(LIST_NOTIFICATION_URL.format(user_id = USERID))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

GET https://e0cfn0j28h.execute-api.us-east-1.amazonaws.com/dev/notification/jupyter_462d26fc
Status: 404
Response: {
  "error": "Notification task not found for the requested user"
}


### Invalid user id

In [5]:
response = requests.get(LIST_NOTIFICATION_URL.format(user_id = 'random-user'))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

GET https://e0cfn0j28h.execute-api.us-east-1.amazonaws.com/dev/notification/random-user
Status: 404
Response: {
  "error": "Notification task not found for the requested user"
}
